# Import Libraries

In [1]:
import requests, base64
import json
import pandas as pd

# (Optional) Import UPC data

In [2]:
data_raw = '___________.csv'

In [3]:
df = pd.read_csv(data_raw)

In [4]:
upc_df = df['gp_upc']

In [5]:
# converting to string dtype 
df['gp_upc'] = df['gp_upc'].astype(str) 
  
# width of output string 
width = 13
  
# calling method and overwriting series 
df['pad_upc'] = df['gp_upc'].str.zfill(width)

In [6]:
upc_lst = list(df['pad_upc'])

# API Creds

In [8]:
# https://developer.kroger.com/
    
base_token_url = 'https://api.kroger.com/v1/connect/oauth2/token'
usrPass = b'<client_id>:<client_secret>'
b64Val = base64.b64encode(usrPass)
b64Val = b64Val.decode("utf-8")

In [9]:
params= {'scope':'product.compact',
        "grant_type": "client_credentials"
        }

headers = {
    "Content-Type": "application/x-www-form-urlencoded",    
    "Authorization": "Basic %s" % b64Val,
  }

In [10]:
token = requests.request("POST", base_token_url, data=params, headers=headers)

In [11]:
b = token.content
r = json.loads(b.decode())

In [12]:
access_token = r['access_token']

# Set initial search criteria

In [13]:
headers = {
    "Accept": "application/json",
    "Authorization": "Bearer %s" % access_token
  }
      
url = "https://api.kroger.com/v1/products"

querystring_one = {
#                 "filter.brand":"Silk"
                "filter.term" :"eggs",
#                 "filter.start": "10",
                "filter.limit" : "200"
              }

In [14]:
results = []

response = requests.request("GET", url, headers=headers, params=querystring_one)

b = response.content
raw = json.loads(b.decode())

limit = int(raw['meta']['pagination']['limit'])
total = int(raw['meta']['pagination']['total'])
start = int(raw['meta']['pagination']['start'])

for i in range(0,len(raw['data'])):
    results.append(raw['data'][i])

# Paginate Remaining Results

There are two types of "limit".  
- how many products to return 
- product number out of the total available

In [21]:
while limit < total:
    
    querystring_loop = {
#                 "filter.brand":"Silk"
                "filter.term" :"eggs",
                "filter.start": str(limit),
                "filter.limit" : "200"
              }
    response = requests.request("GET", url, headers=headers, params=querystring_loop)
    b = response.content
    raw = json.loads(b.decode())
    limit = int(raw['meta']['pagination']['limit'])
    print(limit)

    for i in raw['data']:  
        results.append(i) 

400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6614


In [27]:
list(results[0].keys())

['productId',
 'upc',
 'aisleLocations',
 'brand',
 'categories',
 'description',
 'images',
 'items',
 'itemInformation',
 'temperature']

In [29]:
with open('______.txt', 'w') as outfile:
    json.dump(results, outfile)

In [30]:
# with open('______.txt') as json_file:
#     data = json.load(json_file)